In [43]:
import pandas as pd
from tqdm.notebook import trange, tqdm
from PIL import Image
import pickle
import os

In [2]:
df = pd.read_json(r'dataset/photos.json', lines=True)
df = df.drop(columns = ['business_id', 'caption'])
contador = 0
discarded = 0
for img in df.photo_id.tolist():
    if (contador%30 == 0) and (contador > 0):
        print('Procesadas {:7,} mil fotos de las cuales {} están corruptas'.format(contador, discarded), end='\r');
    contador += 1
    try:
        img = Image.open('dataset/photos/'+img+'.jpg')
        img.verify()
    except:
        # file is corrupt
        df = df.drop(df.loc[df.photo_id == img].index)
        discarded += 1

In [3]:
# dataframe('photo_id', 'label') contiene todas las fotos válidas del dataset
pickle.dump(df, open('checkpoints/valid.pkl', 'wb'))

In [31]:
# checkpoint
df = pickle.load(open('checkpoints/valid.pkl','rb'))

In [32]:
categorias = df.label.value_counts().index

In [33]:
dataset = {'train' : {},
          'test': {},
          'validate': {}}

In [34]:
for categoria in categorias:
    slice_80 = int(len(df.loc[df.label == categoria]) * 0.8)
    slice_10 = int(len(df.loc[df.label == categoria]) * 0.1)
    dataset['train'].update({categoria: df.loc[df.label == categoria][0:slice_80]})
    dataset['test'].update({categoria: df.loc[df.label == categoria][slice_80:slice_80+slice_10]})
    dataset['validate'].update({categoria: df.loc[df.label == categoria][slice_80+slice_10:]})

In [35]:
random_state = 42 # La solución de Douglas Adams para todo
max_photos = 2500

subset_max_photos = {
    'train': int(max_photos * 0.8),
    'test': int(max_photos * 0.1)
}

subset_max_photos.update({
    'validate': max_photos - subset_max_photos['train'] - subset_max_photos['test'] # las que quedan
})

# hacer que los subsets tengan max_photos imagenes
for subset in ['train', 'test', 'validate']:
    for categoria in categorias:
        if len(dataset[subset][categoria]) == subset_max_photos[subset]: # tenemos el numero correcto
            pass            
        elif len(dataset[subset][categoria]) > subset_max_photos[subset]: # demasiadas fotos - resample
            dataset[subset][categoria] = dataset[subset][categoria].sample(n=subset_max_photos[subset],
                                                                           random_state=random_state,
                                                                           replace=False, # solo 1x cada foto
                                                                           ignore_index=True)

In [67]:
dataset_origin_path='dataset/photos'
dataset_output_path='dataset/yelp'
dataset_json_file='dataset/photos.json'

import shutil

# crear estructura de carpetas
if os.path.isdir(dataset_output_path):
    shutil.rmtree(dataset_output_path, ignore_errors=False, onerror=None) # remove tree
    print('{} removed'.format(dataset_output_path))
    
os.mkdir(dataset_output_path)
print('{} created'.format(dataset_output_path))
    
for subset in ['train', 'test', 'validate']:
    os.mkdir(dataset_output_path + '/' + subset)
    print('{} created'.format(dataset_output_path + '/' + subset))
    for categoria in categorias:
        os.mkdir(dataset_output_path + '/' + subset + '/' + categoria)
        print('{} created'.format(dataset_output_path + '/' + subset + '/' + categoria))


dataset/yelp removed
dataset/yelp created
dataset/yelp/train created
dataset/yelp/train/food created
dataset/yelp/train/inside created
dataset/yelp/train/outside created
dataset/yelp/train/drink created
dataset/yelp/train/menu created
dataset/yelp/test created
dataset/yelp/test/food created
dataset/yelp/test/inside created
dataset/yelp/test/outside created
dataset/yelp/test/drink created
dataset/yelp/test/menu created
dataset/yelp/validate created
dataset/yelp/validate/food created
dataset/yelp/validate/inside created
dataset/yelp/validate/outside created
dataset/yelp/validate/drink created
dataset/yelp/validate/menu created


In [68]:
# copiar o generar imagenes

from torchvision import transforms as T

random_state = 42 # La solución de Douglas Adams para todo

from torch import manual_seed
manual_seed(random_state)

import random
random.seed(random_state)

max_photos = 2500

subset_max_photos = {
    'train': int(max_photos * 0.8),
    'test': int(max_photos * 0.1)
}

subset_max_photos.update({
    'validate': max_photos - subset_max_photos['train'] - subset_max_photos['test'] # las que quedan
})

pipeline = T.Compose([
    T.ColorJitter(brightness=.5, hue=.3),
    T.RandomAffine(degrees=(30, 70), scale=(0.9, 1.1))
])


for subset in ['train', 'test', 'validate']:
    for categoria in categorias:
        # copiar
        for img in dataset[subset][categoria].photo_id:
            shutil.copyfile(dataset_origin_path + '/' + img + '.jpg',
                            dataset_output_path + '/' + subset + '/' + categoria + '/' + img + '.jpg')
        
        # generar imagenes si necesario
        if len(dataset[subset][categoria]) < subset_max_photos[subset]: # faltan imagenes

            # determinar cuantas imagenes tendremos que generar
            n_missing = subset_max_photos[subset] - len(dataset[subset][categoria])
            
            # seleccionar imagenes que vamos usar como base
            df_base_images = dataset[subset][categoria].sample(n=n_missing,
                                                               random_state=random_state,
                                                               replace=False, # solo 1x cada foto
                                                               ignore_index=True)
            # abrir original > transformar > guardar imagen transformada > añadir nueva foto al subset
            for img_name in df_base_images.photo_id:
                original_image_path = dataset_origin_path + '/' + img + '.jpg'
                new_image_path = dataset_output_path + '/' + subset + '/' + categoria + '/' + img + '.jpg''_tr.jpg'
                with Image.open(original_image_path) as img:
                    augmented_image = pipeline(img = img)
                    augmented_image.save(new_image_path)
                
                new_img = pd.DataFrame([img_name+'_tr.jpg', 
                                        df.loc[df.photo_id == img_name].label],
                                       columns=['photo_id', 
                                                'label'])
                dataset[subset][categoria] = pd.concat([dataset[subset][categoria], new_img])

ValueError: Shape of passed values is (2, 1), indices imply (2, 2)